# Case 4.1 Fixed Effects estimator

### Write a Matlab function‘FE.m’ that implements the FE estimator
  * Include an option to add time effects


In [6]:
using Distributions: TDist, ccdf

type FE_results
    beta
    SE
    tstat
    pval
end

function FE(Y, X, time_ind; time_effects=false)
    T = Int(maximum(time_ind)-minimum(time_ind)+1) #amount of timesteps
    nT = size(X) #total number of observations
    N = Int(nT[1]/T) #amount of individuals
    k = Int(nT[2]) #amount of parameters to be estimated
    D = kron(eye(N),ones(T,1))
    P_D = D*inv(D'*D)*D'
    M_D = eye(nT[1]) - P_D
    y_dev = M_D*Y #deviation from the mean for y
    x_dev = M_D*X #deviation from the mean for x
    
    if time_effects #time effects
        time_effect = kron(ones(N,1), eye(T)) #per individual there can be an effect per timestep
        x_dev = hcat(x_dev, time_effect)
        k = size(x_dev)[2]
        
    end
    dof = nT[1]-N-k #degrees of freedom left over after having individual effects and estimating the k explanatory variables
    β̂ = inv(x_dev'*x_dev)*x_dev'*y_dev
    residuals = y_dev - x_dev*β̂
    σ̂² = (residuals'*residuals)/dof
    SE = sqrt(diag(σ̂²[1]*inv(x_dev'*x_dev)))
    tstat = β̂ ./ SE
    pval  = 2 * ccdf(TDist(dof), abs(tstat))
    
    FE_results(β̂, SE, tstat, pval)
end

FE (generic function with 1 method)

## Estimate the dynamic CigDem model in levels using the FE estimator

In [2]:
data, header = readcsv("Data_Baltagi.csv", header=true)
println.(header);

state
 year
 P/pack
 pop
 pop16+
 CPI
 ndi/capita
 C/capita
 Pn/pack
 ln C_it
 ln P_it
 ln Pn_it
 ln Y_it


### Because we need to add a lagged value, we will need to delete the rows where year is the minimum value, since this can not be regressed on a lagged value, to do this we will first load the full data and then fill new matrices with the values needed for the lagged regression (hence without year 63 in this example).

In [3]:
Y_temp = data[:, 10]
X_temp = data[:, 11:13]
year_temp = data[:,2]
state_temp = data[:,1]
observations = size(Y_temp)[1]
T = maximum(year_temp) - minimum(year_temp)+1 #amount of timesteps
N = observations/T
#make empty arrays to fill
lagged_obs = Int(observations - N) #removing the N*1 observation to get the amount of lagged observations
Y = zeros(lagged_obs, 1)
Y_lag = zeros(lagged_obs,1)
X = zeros(lagged_obs, size(X_temp)[2])
year = zeros(lagged_obs)
state = zeros(lagged_obs);

In [4]:
#we fill the data-arrays with the first time period dropped
new_i = 1
lag_i = 1
for i = 1:observations
    if year_temp[i] != minimum(year_temp)
        Y[new_i] = Y_temp[i]
        X[new_i,:] = X_temp[i,:]
        year[new_i] = year_temp[i]
        state[new_i] = state_temp[i]
        new_i += 1
    end
    if year_temp[i] != maximum(year_temp)
        Y_lag[lag_i] = Y_temp[i]
        lag_i += 1
    end
end
X = hcat(Y_lag,X);

In [7]:
FE(Y,X,year; time_effects= true).beta

33×1 Array{Float64,2}:
  0.833383  
 -0.298598  
  0.0340454 
  0.100279  
  0.0263246 
  0.0575573 
  0.0406944 
  0.0448442 
  0.0292082 
  0.0257447 
  0.00460896
  0.0502825 
  0.0583618 
  ⋮         
 -0.0434015 
 -0.0524584 
 -0.0488399 
 -0.0443597 
 -0.0149016 
 -0.0179424 
 -0.0207784 
 -0.0264062 
 -0.0269659 
 -0.0284895 
 -0.00926797
  0.0190077 